In [10]:
import requests
import pandas as pd
from io import StringIO
import json
from matplotlib import pyplot
import unicodedata
%matplotlib inline

In [15]:
def get_insee(commune):
    """
    returns the insee code of the commune
    """
    commune_ = unicodedata.normalize('NFD', commune.lower()).encode('ascii', 'ignore').decode("utf-8")
    url = 'https://geo.api.gouv.fr/communes?nom={c}&fields=nom,code,codesPostaux,'\
              'codeDepartement,codeRegion,population&format=json&geometry=centre'\
              .format(c=commune_)
    exists = len(json.loads(requests.get(url).text))
    if exists>0 :
        return json.loads(requests.get(url).text)[0]["code"]
    else:
        print("Commune does not exist")
        return -1

In [16]:
def get_bss(code_commune):
    url =  "https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations?code_commune={c}&format=json&size=20".format(c=code_commune)
    exists = json.loads(requests.get(url).text)["count"]
    if exists>0 :
        data = json.loads(requests.get(url).text)
        bss = [station["code_bss"] for station in data["data"]]
        return bss

    else:
        print("Code commune does not exist")
        return -1

In [21]:
def get_station_piezo(station, start_date=None, end_date=None):
    url = "https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={bss}&date_debut_mesure={d1}&date_fin_mesure={d2}&size=1".format(bss=station, d1=start_date, d2=end_date)
    number = json.loads(requests.get(url).text)["count"]
    if number > 0:
        url = "https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={bss}&date_debut_mesure={d1}&date_fin_mesure={d2}&size={s}".format(bss=station, d1=start_date, d2=end_date, s=number + 1)
        return json.loads(requests.get(url).text)

In [22]:
c = get_insee("Orléans")
c

'45234'

In [23]:
bss  = get_bss(c)
bss

['03635X0545/PZ1',
 '03636X1060/PZ2',
 '03636X1061/PZ3',
 '03636X1062/PZ4',
 '03982X1045/F']

In [24]:
mesure = get_station_piezo(bss[0], start_date = "2019-01-12", end_date = "2019-01-30")
mesure

{'count': 18,
 'first': 'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=03635X0545/PZ1&date_debut_mesure=2019-01-12&date_fin_mesure=2019-01-30&page=1&size=19',
 'last': None,
 'prev': None,
 'next': None,
 'api_version': '1.4.0',
 'data': [{'code_bss': '03635X0545/PZ1',
   'urn_bss': 'http://services.ades.eaufrance.fr/pointeau/03635X0545/PZ1',
   'date_mesure': '2019-01-12',
   'timestamp_mesure': 1547334000000,
   'niveau_nappe_eau': 88.39,
   'mode_obtention': 'Valeur mesurée',
   'statut': 'Donnée contrôlée niveau 2',
   'qualification': 'Correcte',
   'code_continuite': '2',
   'nom_continuite': 'Point lié au point précédent',
   'code_producteur': '293',
   'nom_producteur': 'Service Géologique Régional Centre (293)',
   'code_nature_mesure': None,
   'nom_nature_mesure': None,
   'profondeur_nappe': 4.95},
  {'code_bss': '03635X0545/PZ1',
   'urn_bss': 'http://services.ades.eaufrance.fr/pointeau/03635X0545/PZ1',
   'date_mesure': '2019-01-13',
   'timestamp

In [25]:
import predict_flair
from flair.models import SequenceTagger

batch_size = 4
MODEL_PATH = "/home/maya/Documents/Code/connection_lens/scripts/Flair_NER_tool/stacked-standard-flair-150-wikiner.pt"
tag_type = "label"
model = SequenceTagger.load(MODEL_PATH)

snippets = [[1, "Quel est le niveau de la nappe phréatique à Orléans le 12 janvier 2020?"]]
result = predict_flair.get_entities(snippets, model, tag_type, batch_size)

2021-06-16 22:14:36,359 loading file /home/maya/Documents/Code/connection_lens/scripts/Flair_NER_tool/stacked-standard-flair-150-wikiner.pt


In [26]:
result

{'snippets': [(1,
   {'text': 'Quel est le niveau de la nappe phréatique à Orléans le 12 janvier 2020?',
    'entities': [{'text': 'Orléans',
      'start_pos': 44,
      'end_pos': 51,
      'type': 'LOC',
      'confidence': 0.999824583530426}]})]}